### Hospital Consumer Assessment of Healthcare Providers and Systems (April 2015 -March 2016)

In [1]:
import os
import sys
from pyspark import SparkContext
import matplotlib.pyplot as plt
import numpy as np
from pyspark.sql import SparkSession
%matplotlib inline
%pylab inline
matplotlib.rcParams['figure.figsize'] = (10, 6)

NationalRate = "/Users/pushparajparab/Desktop/Hospital_Revised_Flatfiles/HCAHPS__Hospital.csv"
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(NationalRate)

df.registerTempTable('Mytable')


query = " SELECT Hospital_Name,HCAHPS_Answer_Description,Patient_Survey_Star_Rating"
query += " FROM MyTable M"
query += " WHERE HCAHPS_Answer_Description LIKE '%star rating%'"
query += " AND Patient_Survey_Star_Rating != 'Not Available'"
query += " order by M.Hospital_Name,HCAHPS_Answer_Description"
distint = spark.sql(query)
# distint.show(20, False)
# Complications = "/Users/pushparajparab/Desktop/Hospital_Revised_Flatfiles/Complications_Hospital.csv";
# df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(Complications)
# df.registerTempTable('Data')


query = " SELECT HCAHPS_Answer_Description,Patient_Survey_Star_Rating,count(Patient_Survey_Star_Rating) as count"
query += " FROM MyTable M"
query += " WHERE HCAHPS_Answer_Description LIKE '%star rating%'"
query += " AND HCAHPS_Answer_Description NOT LIKE '%Recommend%'"
query += " AND HCAHPS_Answer_Description NOT LIKE '%Summary%'"
query += " AND HCAHPS_Answer_Description NOT LIKE '%Overall%'"
query += " AND Patient_Survey_Star_Rating != 'Not Available'"
query += " group by M.Patient_Survey_Star_Rating,HCAHPS_Answer_Description"
query += " order by count(Patient_Survey_Star_Rating) desc"
distint = spark.sql(query)
# distint.show(20, False)

_size =[]
_text = []
_x=[]
_y=[]
_dataList = distint.collect()
for i in range(len(_dataList)):
    _x.append(_dataList[i][0])
    _y.append(_dataList[i][1])
    _size.append(_dataList[i][2]*0.08)
    _text.append('Count: '+ str(_dataList[i][2]))


Populating the interactive namespace from numpy and matplotlib


In [2]:
import plotly
# Set Plotly credetials

plotly.tools.set_credentials_file(username='pparab', api_key='jgQSn7NrsQAUnpx26Aks')
import plotly.plotly as py
import plotly.tools as plotly_tools
import plotly.graph_objs as go
from plotly.graph_objs import *
 #Sign in the plotly API
py.sign_in("pparab", "jgQSn7NrsQAUnpx26Aks")

trace0 = go.Scatter(
    x=_x,
    y=_y,
    text=_text,
    mode='markers',
    marker=dict(
        size=_size,
    )
)

data = [trace0]
py.iplot(data, filename='bubblechart-size')

In [3]:
import ipywidgets as widgets
from IPython.display import clear_output 



query = " SELECT DISTINCT State"
query += " FROM MyTable D"
query += " ORDER BY State"
plt.style.use('fivethirtyeight')#assign the style for the plot
distint = spark.sql(query)
_dataList = distint.collect()
_states =[]
_selectedState = ""
for i in range(len(_dataList)):
    _states.append(_dataList[i][0])

select = widgets.Dropdown(
    options=_states,
#      value='AZ',
    description='State:',
    disabled=False
)
display(select)

o_query = " SELECT HCAHPS_Answer_Description,Patient_Survey_Star_Rating,count(Patient_Survey_Star_Rating) as count"
o_query += " FROM MyTable M"
o_query += " WHERE HCAHPS_Answer_Description LIKE '%star rating%'"
o_query += " AND HCAHPS_Answer_Description NOT LIKE '%Recommend%'"
o_query += " AND HCAHPS_Answer_Description NOT LIKE '%Summary%'"
o_query += " AND HCAHPS_Answer_Description NOT LIKE '%Overall%'"
o_query += " AND Patient_Survey_Star_Rating != 'Not Available'"

distint = spark.sql(query)
# distint.show(20, False)

_size =[]
_text = []
_x=[]
_y=[]
_SelectedState = ["NA"]

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output()
        
        del _size[:]
        del _text[:]
        del _x[:]
        del _y[:]
        _SelectedState[0] = change['new']
        _selectedState = change['new']
        query = o_query + " AND State = '"
        query += _selectedState
        query += "' group by M.Patient_Survey_Star_Rating,HCAHPS_Answer_Description"
        query += " order by count(Patient_Survey_Star_Rating) desc"
#         print(query)
        results = spark.sql(query);
#         results.show(20, False)

        _dataList = results.collect()
        for i in range(len(_dataList)):
            _x.append(_dataList[i][0])
            _y.append(_dataList[i][1])
            _size.append(_dataList[i][2])
            _text.append('Count: '+ str(_dataList[i][2]))


select.observe(on_change)
clear_output()

In [4]:
print("Plotting for:" + _SelectedState[0])
trace0 = go.Scatter(
    x=_x,
    y=_y,
    text=_text,
    mode='markers',
    marker=dict(
        size=_size
    )
)

data = [trace0]
py.iplot(data, filename='bubblechart-size')



Plotting for:NC


# Rating below 2

In [5]:
u_query = "SELECT Hospital_Name,HCAHPS_Answer_Description FROM MyTable D"
u_query += " WHERE HCAHPS_Answer_Description LIKE '%star rating%'"
u_query += " AND HCAHPS_Answer_Description NOT LIKE '%Recommend%'"
u_query += " AND HCAHPS_Answer_Description NOT LIKE '%Summary%'"
u_query += " AND HCAHPS_Answer_Description NOT LIKE '%Overall%'"
u_query += " AND Patient_Survey_Star_Rating != 'Not Available'"
u_query += " AND (Patient_Survey_Star_Rating = 1 OR Patient_Survey_Star_Rating = 2 )"
u_query += " AND State = '"+_SelectedState[0]+"'"
# u_query += " and Hospital_Name='TULANE MEDICAL CENTER'" 
u_query += " ORDER BY Hospital_Name ASC"
u_result = spark.sql(u_query)
u_result.registerTempTable('WTN')
# u_result.show(20, False)

_x=[]
_y=[]

_dataList = u_result.collect();
for i in range(len(_dataList)):
            _x.append(_dataList[i][0])
            _y.append(_dataList[i][1])
            
trace = go.Scatter(
    x = _x,
    y = _y,
    hoveron = 'points',
    mode = 'markers',
    marker =dict( 
        size = 8,
        color = '#FF7070'
    )
)

data = [trace]
layout = go.Layout(
    autosize=False,
    width=900,
    height=900,
    margin=go.Margin(
        l=340,
        r=200,
        b=300,
        t=250,
        pad=4
    ),
    paper_bgcolor='#fff',
    plot_bgcolor='#fff'
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-scatter_HCAPS_Low')

# Rating more then 4

In [6]:
u_query = "SELECT Hospital_Name,HCAHPS_Answer_Description FROM MyTable D"
u_query += " WHERE HCAHPS_Answer_Description LIKE '%star rating%'"
u_query += " AND HCAHPS_Answer_Description NOT LIKE '%Recommend%'"
u_query += " AND HCAHPS_Answer_Description NOT LIKE '%Summary%'"
u_query += " AND HCAHPS_Answer_Description NOT LIKE '%Overall%'"
u_query += " AND Patient_Survey_Star_Rating != 'Not Available'"
u_query += " AND (Patient_Survey_Star_Rating = 4 OR Patient_Survey_Star_Rating = 5 )"
u_query += " AND State = '"+ _SelectedState[0] +"'"
# u_query += " and Hospital_Name='TULANE MEDICAL CENTER'" 
u_query += " ORDER BY Hospital_Name ASC"
u_result = spark.sql(u_query)
u_result.registerTempTable('BTN')
# u_result.show(20, False)

_x=[]
_y=[]

_dataList = u_result.collect();
for i in range(len(_dataList)):
            _x.append(_dataList[i][0])
            _y.append(_dataList[i][1])
            
trace = go.Scatter(
    x = _x,
    y = _y,
    hoveron = 'points',
    mode = 'markers',
    marker =dict( 
        size = 8,
        color = '#008000'
    )
)

data = [trace]
layout = go.Layout(
    autosize=False,
    width=900,
    height=900,
    margin=go.Margin(
        l=340,
        r=200,
        b=300,
        t=250,
        pad=4
    ),
    paper_bgcolor='#fff',
    plot_bgcolor='#fff'
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-scatter_HCAPS_High')